# Remesas

In [1]:
# Importar librerías
import pandas as pd
import os
from datetime import datetime, timedelta
import pyodbc
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle

In [2]:
input("RECORDATORIO: ¿Ya cambiaste el timedelta del Lunes o Miércoles/Viernes?. Presiona ENTER para continuar")
input("RECORDATORIO: ¿Ya actualizaste el número de remesa correspondiente?. Presiona ENTER para continuar")

RECORDATORIO: ¿Ya cambiaste el timedelta del Lunes o Miércoles/Viernes?. Presiona ENTER para continuar
RECORDATORIO: ¿Ya actualizaste el número de remesa correspondiente?. Presiona ENTER para continuar


''

### IMPORTANTE: Actualiza los datos de abajo cada día

In [3]:
# DEFINIR FECHAS
# remesa de los lunes editar timedelta(days=2)
# remesa de miercoles y viernes dejar timedelta(days=1)
#dia_hoy = (datetime.now() - timedelta(days=6)).strftime('%d%m%Y') # Utilizar en caso de remesa omitida

dia_hoy = datetime.now().strftime('%d%m%Y')
dia_ayer = (datetime.now() - timedelta(days=1)).strftime('%d%m%Y')

# DEFINIR NUMERO DE REMESAS
num_bradescard = '289'
num_copayment = '561'
num_prosa = '792'

In [4]:
# Ruta de la carpeta de descargas (ahí se genera el archivo)
carpeta_descargas = os.path.join(os.path.expanduser('~'), 'Downloads')

# Nombre del archivo Excel que deseas crear
pre_remesa = f'Pre remesa {dia_hoy}.xlsx'

# Crear la ruta completa del archivo Pre remesa
path_archivo = os.path.join(carpeta_descargas, pre_remesa)

# Crear un DataFrame (df)
df = pd.DataFrame()

In [5]:
# Nombre del archivo Excel que deseas crear
bradescard = f'Lote de aclaraciones {num_bradescard}.xlsx'
# Crear la ruta completa del archivo Lote de aclaraciones
path_archivo_b = os.path.join(carpeta_descargas, bradescard)

# Nombre del archivo Excel que deseas crear
copayment = f'Remesa {num_copayment}.xlsx'
# Crear la ruta completa del archivo Remesa copayment
path_archivo_c = os.path.join(carpeta_descargas, copayment)

# Nombre del archivo Excel que deseas crear
prosa = f'Remesa {num_prosa}.xlsx'
# Crear la ruta completa del archivo Remesa prosa
path_archivo_p = os.path.join(carpeta_descargas, prosa)

In [6]:
# Conexión a la base de datos
server = "MP-VW-DB-021"  # Tu servidor
database = "BONIFINMED"  # Tu base de datos

conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

try:
    # Mensaje que indica si la conexión es exitosa
    connection = pyodbc.connect(conn_str)
    print("Conexión exitosa")
    
    # Calcular las fechas de inicio y fin de la consulta
    hoy = datetime.now()
    hace_20_dias = hoy - timedelta(days=20)

    fecha_hoy_str = hoy.strftime('%Y-%m-%d')
    fecha_hace_20_dias_str = hace_20_dias.strftime('%Y-%m-%d')

    # Consulta de SQL Procesos_Bonificación_Inmediata_VF 2.0
    consulta_sql = f'''
    WITH FilasEnumeradas AS (
      SELECT
        CASE
          WHEN tblTransaccionHistorica.nombreArchivo <> ('SitioWeb') THEN 'Falcon'
          ELSE tblTransaccionHistorica.nombreArchivo
        END AS Ingreso,
        tblTransaccionHistorica.duplicidad510,
        tblTransaccionHistorica.nombreArchivo,
        tblTransaccionHistorica.tarjeta, 
        tblTransaccionHistorica.ID_Auto, 
        tblTransaccionHistorica.importeTran, 
        tblTransaccionHistorica.fechaTran, 
        tblProsaTemp.Hr, 
        tblProsaTemp.estatusCompra,
        tblTransaccionReglaBonificacion.importe AS Monto_Bonificado, 
        CASE
          WHEN TIPO_TRANSACCION IN ('01') THEN 'Transacción compra'
          WHEN TIPO_TRANSACCION IN ('21') THEN 'Transacción Devolución'
          WHEN TIPO_TRANSACCION IS NULL AND tblFlujoLevantamiento2.tipoFraude IN ('ATM','Transacción ATM con categoria diferente a ATM') THEN 'Disposición en efectivo'
          WHEN TIPO_TRANSACCION IS NULL AND tblTransaccionReglaBonificacion.ticket IS NULL THEN 'No posteado'
          WHEN TIPO_TRANSACCION IS NULL AND reglaICE_ECI IN ('=') THEN 'Transacción compra'
          ELSE TIPO_TRANSACCION
        END AS Tipo_Transacción,
        tblTransaccionReferencia.referencia AS Referencia,
        tblProsaTemp.TERM_CNTRY_CDE AS Pais,
        tblTransaccionHistorica.Nombre_Cmr AS Nombre_Comercio, 
        SUBSTRING(tblProsaTemp.PT_SRV_ENTRY_MDE, 1, 2) AS PEM, 
        (tblFlujoLevantamiento2.reglaICE_ECI + tblFlujoLevantamiento2.iCE_ECI) AS ICE,
        CASE
          WHEN tblFlujoLevantamiento2.tipoFraude IS NULL THEN 'No posteado'
          ELSE tblTransaccionReglaBonificacion.ticket
        END AS Ticket_GLPI,
        [dbo].[tmpTranFolio].FOLIO_BI,
        [dbo].[tmpTranFolio].TIPO_ACLARACIÓN,
        CASE
          WHEN tblFlujoLevantamiento2.tipoFraude IS NULL THEN 'No posteada'
          ELSE tblFlujoLevantamiento2.tipoFraude
        END AS Categoria_Regla,
        CASE
          WHEN descripcionBonificar IS NULL THEN 'No se bonifica'
          ELSE descripcionBonificar
        END AS Regla_Asignada,
        CASE
          WHEN tblTransaccionReglaBonificacion.tipobonificacion IS NULL THEN 'No se Bonifica'
          ELSE tblTransaccionReglaBonificacion.tipobonificacion
        END AS Tipo_Bonificación,
        CASE
          WHEN tblTransaccionReglaBonificacion.estatusAsignado IS NULL THEN 'No posteada'
          ELSE tblTransaccionReglaBonificacion.estatusAsignado
        END AS Estatus_Asignado,
        tblProsaTemp.fechaAct AS Fecha_Ult_Estatus_Act,
        CASE
          WHEN tblTransaccionReglaBonificacion.aplicaTC40 IS NULL THEN 'NO'
          ELSE tblTransaccionReglaBonificacion.aplicaTC40
        END AS Aplica_TC40,
        CASE
          WHEN tblTransaccionReglaBonificacion.aplicaRemesa IS NULL THEN 'NO'
          ELSE tblTransaccionReglaBonificacion.aplicaRemesa
        END AS Aplica_Remesa,
        tblTransaccionHistorica.fechaAct AS Fecha_marcado, 
        [BONIFICACION].[dbo].[tblFolio].fechaAlta,
        BONIFICACION.dbo.tblFolio.comentario,
        BONIFICACION.dbo.tblFolio.statementDay,
        tblTransaccionReglaBonificacion.fechaAct AS Fecha_Proceso,
        tblRemesa2.nombreArchivo AS Nombre_Remesa,
        tblTransaccionReglaBonificacion.estatusTicket,
        ROW_NUMBER() OVER (PARTITION BY tblTransaccionHistorica.tarjeta, tblTransaccionHistorica.id_auto ORDER BY tblTransaccionHistorica.fechaTran DESC) AS NumFila
      FROM
        tblTransaccionHistorica WITH (NOLOCK) 
      LEFT JOIN 
        tblTransaccionReglaBonificacion WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = tblTransaccionReglaBonificacion.tarjeta AND tblTransaccionHistorica.ID_Auto = tblTransaccionReglaBonificacion.apprv
      LEFT JOIN 
        tbl510POSHistorial WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = tbl510POSHistorial.NUM_CUENTA AND tblTransaccionHistorica.ID_Auto = tbl510POSHistorial.NUM_AUT
      LEFT JOIN 
        tblFlujoLevantamiento2 WITH (NOLOCK) ON tblTransaccionReglaBonificacion.idFlujoLevantamiento = tblFlujoLevantamiento2.idFlujoLevantamiento
      LEFT JOIN 
        tbl510ATMHistorial WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = tbl510ATMHistorial.NumeroDTarjeta AND tblTransaccionHistorica.ID_Auto = tbl510ATMHistorial.Autorización
      LEFT JOIN
        tblProsaTemp WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = tblProsaTemp.NUM AND tblTransaccionHistorica.ID_Auto = tblProsaTemp.APPRV_CDE
      LEFT JOIN  
        [dbo].[tmpTranFolio] WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = [dbo].[tmpTranFolio].NUM AND tblTransaccionHistorica.ID_Auto = [dbo].[tmpTranFolio].autorizacion
      LEFT JOIN 
        tblTransaccionReferencia WITH (NOLOCK) ON tblTransaccionHistorica.tarjeta = tblTransaccionReferencia.tarjeta AND tblTransaccionHistorica.ID_Auto = tblTransaccionReferencia.apprv
      LEFT JOIN 
        BONIFICACION.dbo.tblFolio WITH (NOLOCK) ON [dbo].[tmpTranFolio].FOLIO_BI = BONIFICACION.dbo.tblFolio.folio
      LEFT JOIN 
        tblRemesa2 WITH (NOLOCK) ON tblTransaccionReglaBonificacion.tarjeta = tblRemesa2.TARJETA AND tblTransaccionReglaBonificacion.apprv = tblRemesa2.AUT 
    )
    SELECT
      CASE
        WHEN Nombre_Comercio = 'COMPRA BRADES WEB APP' THEN 'Weconnect NO ENVIAR A REMESA'
        ELSE 'Flujo Normal'
      END AS Marcador_Weconnect,
      Ingreso,
      Fecha_Proceso,
      duplicidad510,
      fechaAlta AS 'Fecha Interacción',
      tarjeta,
      ID_Auto,
      importeTran,
      Monto_Bonificado,
      fechaTran,
      Hr,
      Referencia,
      Pais,
      Nombre_Comercio,
      PEM,
      ICE,
      CASE 
        WHEN Ticket_GLPI IS NULL THEN 'Pendiente Alta de ticket'
        ELSE Ticket_GLPI 
      END AS Ticket_GLPI,
      estatusTicket,
      Folio_BI,
      TIPO_ACLARACIÓN,
      Categoria_Regla,
      Regla_Asignada,
      CASE
        WHEN Estatus_Asignado = 'Sin regla asignada' THEN 'No se Bonifica'
        ELSE Tipo_Bonificación
      END AS Tipo_Bonificación,
      Estatus_Asignado,
      Aplica_TC40,
      Aplica_Remesa,
      Nombre_Remesa,
      Comentario,
      statementDay,
      CASE
        WHEN Estatus_Asignado IN ('Bonificadas','Sin regla asignada','En análisis de fraude') AND Aplica_Remesa IN ('SI') THEN 'Cuenta en análisis, en espera de contracargo/dictamen'
        WHEN Estatus_Asignado IN ('En análisis de fraude') AND ICE IN ('=5') THEN 'Validar VCAS/en espera de dictamen' 
        WHEN Estatus_Asignado IN ('Quebranto') THEN 'Validar Accesorios y Cerrar ticket a favor del cliente'
        WHEN Estatus_Asignado IN ('No posteada') THEN 'Transacción No posteada'
        WHEN Regla_Asignada LIKE ('%total%') THEN 'Devolución por parte del comercio, cerrar ticket'
        WHEN Categoria_Regla IN ('Me la robaron/la extravíe') THEN 'Bloqueo R, Improcedente por políticas Internas'
        WHEN Categoria_Regla IN ('No hice el trámite de esa tarjeta (Robo de identidad)','Cancelación no atendida','Doble emboso','Pre aprobados','Primeras compras en agregadores','Reposición/Adicional no reconocida','Tarjeta no entregada') THEN 'Realizar Predictamen, Canalizar con Fraudes'
      END AS Dictamen
    FROM
      FilasEnumeradas WITH (NOLOCK)
    WHERE 
      numFila = 1  
      AND fechaAlta >= '{fecha_hace_20_dias_str}' AND fechaAlta < '{fecha_hoy_str} 09:00:00.000'
    ORDER BY [Fecha Interacción] DESC
    '''
    
    # Ejecutar la consulta y cargar los datos en df
    df = pd.read_sql(consulta_sql, connection)
    
    # Convertir todas las columnas a texto (string)
    df = df.astype(str)

except Exception as e:
    print("Error al ejecutar la consulta o la conexión: ", e)

finally:
    # Cerrar la conexión
    connection.close()
    print("Conexión cerrada")

Conexión exitosa


C:\Users\yordi.ruelas\AppData\Local\Temp\ipykernel_9500\2019052220.py:171: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(consulta_sql, connection)


Conexión cerrada


In [7]:
# Crear la columna 'Llave' concatenando 'tarjeta' y 'ID_Auto'
df['Llave'] = df['tarjeta'] + df['ID_Auto']
df['Llave'] = df['Llave'].astype(str)

# Crear la columna 'Escenario'
df['Escenario'] = ''

# Reorganizar las columnas para que 'Llave' y 'Escenario' estén al principio
df = df[['Llave', 'Escenario'] + [col for col in df.columns if col not in ['Llave', 'Escenario']]]

In [8]:
# Especifica la ruta completa al archivo de Excel Consolidado BI 2025
ruta_archivo_excel = r'\\celerrabr\Compartido\Entre_Sesiones\D7627S086\Aclaracion y Prev de Fraudes\MIS\Remesas\Remesas Historico\2025\Consolidado_Pre_Remesas_2025.xlsx'  # Cambia esta ruta según sea necesario
hoja_nombre = 'Consolidado BI 2025'

# Cargar la hoja en un DataFrame llamado df_consolidado
df_consolidado = pd.read_excel(ruta_archivo_excel, sheet_name=hoja_nombre)
df_consolidado['Llave tarj & Aut'] = df_consolidado['Llave tarj & Aut'].astype(str)

# Crear llave en df_consolidado
df['Llave'] = df['Llave'].str.strip()
df_consolidado['Llave tarj & Aut'] = df_consolidado['Llave tarj & Aut'].str.strip()

df['Llave'] = df['Llave'].str.replace(r'\s+', '', regex=True)
df_consolidado['Llave tarj & Aut'] = df_consolidado['Llave tarj & Aut'].str.replace(r'\s+', '', regex=True)

df = df.drop_duplicates(subset=['Llave'])
df_consolidado = df_consolidado.drop_duplicates(subset=['Llave tarj & Aut'])

# Usar una máscara para encontrar todas las filas que contienen la palabra 'prueba' (ignora mayúsculas y minúsculas)
# mask_prueba = df['Comentario'].str.contains('prueba', case=False, na=False)

# Actualizar la columna Escenario en las filas donde se encuentra 'prueba' en la columna Comentario
#df.loc[mask_prueba, 'Escenario'] = 'Ticket prueba productiva'

# Hacer el cruce
df = df.merge(df_consolidado[['Llave tarj & Aut']], 
              left_on='Llave', right_on='Llave tarj & Aut', 
              how='left', indicator=True)

# Marcamos como "Ya trabajados" solo las filas que cruzaron con el consolidado
df['Escenario'] = df.apply(lambda row: 'Ya trabajados' if row['_merge'] == 'both' else row['Escenario'], axis=1)

# Eliminamos la columna '_merge' que se genera por el merge
df = df.drop(columns=['_merge'])

In [9]:
# Escenario Weconnect
df['Escenario'] = df.apply(
    lambda row: 'Weconnect' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['Marcador_Weconnect'] in ['Weconnect NO ENVIAR A REMESA']
    ) else row['Escenario'],
    axis=1
)

# Escenario FNZ*ESC XHIDEL COMERCI
df['Escenario'] = df.apply(
    lambda row: 'FNZ*ESC XHIDEL COMERCI' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['Nombre_Comercio'] in ['FNZ*ESC XHIDEL COMERCI']
    ) else row['Escenario'],
    axis=1
)

# Escenario No posteado
df['Escenario'] = df.apply(
    lambda row: 'No posteado' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['Ticket_GLPI'] in ['No posteado']
    ) else row['Escenario'],
    axis=1
)

# Escenario Pendiente Alta de ticket
df['Escenario'] = df.apply(
    lambda row: 'Pendiente Alta de ticket' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['Ticket_GLPI'] in ['Pendiente Alta de ticket']
    ) else row['Escenario'],
    axis=1
)

# Escenario Se trabaja en archivo siguiente
df['Escenario'] = df.apply(
    lambda row: 'Se trabaja en archivo siguiente' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['Ticket_GLPI'] in ['']
    ) else row['Escenario'],
    axis=1
)

In [10]:
# Ruta de archivo de las remesas
ruta_remesa = r'\\celerrabr\Reportes\BonificacionInmediata\Archivos\Remesa'

# Nombre de las remesas a descargar
remesa_hoy = f'''Remesa_{dia_hoy}.xls'''
remesa_ayer = f'''Remesa_{dia_ayer}.xls'''

# Descargar remesas
ruta_remesa_hoy = os.path.join(ruta_remesa, remesa_hoy)
df_remesa_hoy = pd.read_excel(
    ruta_remesa_hoy,
    dtype={
        'AUT': str,
        'MCC': str,
        'IMPORTE': str,
        'PEM': str,
        'AFILIACION': str,
        'ICE': str,
        'TOKEN_Q2': str
    }, 
    engine='openpyxl'
)

ruta_remesa_ayer = os.path.join(ruta_remesa, remesa_ayer)
df_remesa_ayer = pd.read_excel(
    ruta_remesa_ayer,
    dtype={
        'AUT': str,
        'MCC': str,
        'IMPORTE': str,
        'PEM': str,
        'AFILIACION': str,
        'ICE': str,
        'TOKEN_Q2': str
    }, 
    engine='openpyxl'
)

# unir las dos remesas en un mismo DataFrame
df_remesas = pd.concat([df_remesa_hoy, df_remesa_ayer], ignore_index=True)
df_remesas = df_remesas.astype(str)

In [11]:
# Crear Llave para df_remesas
df_remesas['Llave'] = df_remesas['TARJETA'] + df_remesas['AUT']
df_remesas['Llave'] = df_remesas['Llave'].astype(str)

# Añadir columnas a df_remesas
df_remesas['Comentario y # remesa'] = ''
df_remesas['Proveedor'] = ''

df_remesas = df_remesas[['Llave', 'Comentario y # remesa', 'Proveedor'] + [col for col in df_remesas.columns if col not in ['Llave', 'Comentario y # remesa', 'Proveedor']]]

llaves_remesa = df_remesas['Llave'].unique()  # Lista de llaves únicas en df_remesas

In [12]:
# Escenario Enviados a remesa
df['Escenario'] = df.apply(
    lambda row: 'Enviados a remesa' if (pd.isna(row['Escenario']) or row['Escenario'] == '') and row['Llave'] in llaves_remesa else row['Escenario'], 
    axis=1
)

# Escenario Monto menor
df['Escenario'] = df.apply(
    lambda row: 'Monto menor' if (
        pd.isna(row['Escenario']) or row['Escenario'] == '' # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP', 'Devolución no aplicada', 'No pasó y pagó con otro medio'] # Condiciones en TIPO_ACLARACIÓN
        and 'monto menor' in str(row['Regla_Asignada']).lower() # Condición en Regla_Asignada (insensible a mayúsculas)
    ) else row['Escenario'],
    axis=1
)

# Escenario ICE 5
df['Escenario'] = df.apply(
    lambda row: 'ICE 5' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP']  # Condiciones en TIPO_ACLARACIÓN
        and row['ICE'] == '=5'  # Condición en ICE
    ) else row['Escenario'],
    axis=1
)

# Escenario Devolución Total
df['Escenario'] = df.apply(
    lambda row: 'Devolución Total' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP']  # Condiciones en TIPO_ACLARACIÓN
        and row['Regla_Asignada'] == 'Si se identifica que la devolución es total '  # Condición en ICE
    ) else row['Escenario'],
    axis=1
)

# Escenario Bloqueo R
df['Escenario'] = df.apply(
    lambda row: 'Bloqueo R' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Me la robaron/la extravíe']  # Condiciones en TIPO_ACLARACIÓN
    ) else row['Escenario'],
    axis=1
)

# Escenario TNR
df['Escenario'] = df.apply(
    lambda row: 'TNR' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Cancelación no atendida', 'Reposición/Adicional no reconocida', 'Tarjeta no entregada', 'No hice el trámite de esa tarjeta (Robo de identidad)', 'Doble emboso', 'Pre aprobados', 'Cargos adicionales no autorizados por mí']  # Condiciones en TIPO_ACLARACIÓN
    ) else row['Escenario'],
    axis=1
)

# Escenario Aclaración Incorrecta
df['Escenario'] = df.apply(
    lambda row: 'Aclaración Incorrecta' if (
        pd.isna(row['Escenario']) or row['Escenario'] == ''  # Escenario está vacío
        and row['TIPO_ACLARACIÓN'] in ['Cargo duplicado']  # Condiciones en TIPO_ACLARACIÓN
    ) else row['Escenario'],
    axis=1
)

In [13]:
# Eliminar PRUEBAS de remesas
#filtro_prueba = df_remesas['COMENTARIO_TKT'].str.contains(r'prueba|peurba', case=False, na=False)

#df_remesas = df_remesas[~filtro_prueba]

# Eliminar Weconnect de remesas
llaves_weconnect = df[df['Escenario'] == 'Weconnect']['Llave']

df_remesas = df_remesas[~df_remesas['Llave'].isin(llaves_weconnect)]

# Eliminar FNZ*ESC XHIDEL COMERCI de remesas
llaves_xhideli = df[df['Escenario'] == 'FNZ*ESC XHIDEL COMERCI']['Llave']

df_remesas = df_remesas[~df_remesas['Llave'].isin(llaves_xhideli)]

In [14]:
#Corregir los Token_Q2 que sean igual a 00

#Lee el Catalogo de Q2 que esta en descargas formato excel
ruta_descargas = os.path.join(os.path.expanduser("~"), "Downloads")
nombre_archivo = "Validacion Q2-0.xlsx"
ruta_completa = os.path.join(ruta_descargas, nombre_archivo)

# Leer Excel y convertirlo a diccionario
df_catalogo = pd.read_excel(ruta_completa, dtype={0: str, 1: str})
dic = dict(zip(df_catalogo.iloc[:, 0], df_catalogo.iloc[:, 1]))

df_token_cero = df_remesas[df_remesas["TOKEN_Q2"] == "00"]
cantidad = len(df_token_cero)


#Cambiar el Q2 00 por el que corresponda
df_remesas.loc[df_remesas['TOKEN_Q2'] == '00', 'TOKEN_Q2'] = \
df_remesas.loc[df_remesas['TOKEN_Q2'] == '00', 'AFILIACION'].map(dic).fillna('00')

df_token_cero = df_remesas[df_remesas["TOKEN_Q2"] == "00"]
cantidad_nueva = len(df_token_cero)
cantidad = cantidad - cantidad_nueva
cantidad = str(cantidad)

print(cantidad + " Casos con 00 remplazados")



51 Casos con 00 remplazados


In [15]:
# ** Corrección de error TIPO_BLOQUEO > Fraude Digitado, COD_PAIS > MX, TOKEN_Q2 > 03, PEM > != 01 10, TIPO_BLOQUEO = Clonación
df_remesas['TIPO_BLOQUEO'] = df_remesas.apply(
    lambda row: 'Clonación' if (
        row['TIPO_BLOQUEO'] in ['Fraude Digitado']
        and row['COD_PAIS'] in ['MX']
        and row['TOKEN_Q2'] in ['03', '04']
        and row['PEM'] in ['02', '05', '07', '80', '90']
    ) else row['TIPO_BLOQUEO'],
    axis=1
)

# ** Correccion de error Compra no reconocida y PEM None = Compra no reconocida y PEM 01
df['PEM'] = df.apply(
    lambda row: '01' if (
        row['TIPO_ACLARACIÓN'] in ['Compra no reconocida']
        and row['PEM'] in ['None']
    ) else row['PEM'],
    axis=1
)

In [16]:
# Proveedor BRADESCARD
df_remesas['Proveedor'] = df_remesas.apply(
    lambda row: 'BRADESCARD' if (
        pd.isna(row['Proveedor']) or row['Proveedor'] == ''
        and row['TIPO_BLOQUEO'] in ['Fraude Digitado']
        and row['COD_PAIS'] in ['MX']
        and row['TOKEN_Q2'] in ['02', '08', '09']
    ) else row['Proveedor'],
    axis=1
)
# Comentario y # remesa Lote de aclaraciones XXX
df_remesas['Comentario y # remesa'] = df_remesas.apply(
    lambda row: f'Lote de aclaraciones {num_bradescard}' if (
        pd.isna(row['Comentario y # remesa']) or row['Comentario y # remesa'] == ''
        and row['TIPO_BLOQUEO'] in ['Fraude Digitado']
        and row['COD_PAIS'] in ['MX']
        and row['TOKEN_Q2'] in ['02', '08', '09']
    ) else row['Comentario y # remesa'],
    axis=1
)

# Proveedor y Comentario y # remesa PEM 07
df_remesas['Proveedor'] = df_remesas.apply(
    lambda row: 'PEM 07' if (
        pd.isna(row['Proveedor']) or row['Proveedor'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación']
        and row['COD_PAIS'] in ['MX']
        and row['PEM'] in ['07']
    ) else row['Proveedor'],
    axis=1
)
df_remesas['Comentario y # remesa'] = df_remesas.apply(
    lambda row: 'PEM 07' if (
        pd.isna(row['Comentario y # remesa']) or row['Comentario y # remesa'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación']
        and row['COD_PAIS'] in ['MX']
        and row['PEM'] in ['07']
    ) else row['Comentario y # remesa'],
    axis=1
)

# Proveedor y Comentario y # remesa Clonación Internacional
df_remesas['Proveedor'] = df_remesas.apply(
    lambda row: 'Clonación internacional' if (
        pd.isna(row['Proveedor']) or row['Proveedor'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación']
        and row['COD_PAIS'] not in ['MX']
    ) else row['Proveedor'],
    axis=1
)
df_remesas['Comentario y # remesa'] = df_remesas.apply(
    lambda row: 'Clonación internacional' if (
        pd.isna(row['Comentario y # remesa']) or row['Comentario y # remesa'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación']
        and row['COD_PAIS'] not in ['MX']
    ) else row['Comentario y # remesa'],
    axis=1
)

# Proveedor PROSA
df_remesas['Proveedor'] = df_remesas.apply(
    lambda row: 'PROSA' if (
        pd.isna(row['Proveedor']) or row['Proveedor'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación', 'ATM']
        and row['COD_PAIS'] in ['MX', 'None']
    ) else row['Proveedor'],
    axis=1
)
# Comentario y # remesa XXX (prosa)
df_remesas['Comentario y # remesa'] = df_remesas.apply(
    lambda row: num_prosa if (
        pd.isna(row['Comentario y # remesa']) or row['Comentario y # remesa'] == ''
        and row['TIPO_BLOQUEO'] in ['Clonación', 'ATM']
        and row['COD_PAIS'] in ['MX', 'None']
    ) else row['Comentario y # remesa'],
    axis=1
)

# Proveedor COPAYMENT
df_remesas['Proveedor'] = df_remesas.apply(
    lambda row: 'COPAYMENT' if (
        pd.isna(row['Proveedor']) or row['Proveedor'] == ''
    ) else row['Proveedor'],
    axis=1
)
# Comentario y # remesa XXX (copayment)
df_remesas['Comentario y # remesa'] = df_remesas.apply(
    lambda row: num_copayment if (
        pd.isna(row['Comentario y # remesa']) or row['Comentario y # remesa'] == ''
    ) else row['Comentario y # remesa'],
    axis=1
)

In [17]:
# Filtrar los registros de df_remesas donde el proveedor es PEM 07
df_remesas_pem07 = df_remesas[df_remesas['Proveedor'] == 'PEM 07']
# Actualizar la columna 'Escenario' en df cuando las 'Llave' coinciden y el proveedor es PEM 07
df['Escenario'] = df.apply(
    lambda row: 'PEM 07' if row['Llave'] in df_remesas_pem07['Llave'].values and row['Escenario'] not in ['Ya trabajados'] else row['Escenario'], axis=1)
# Eliminar los registros de df_remesas donde el proveedor es PEM 07
df_remesas = df_remesas[df_remesas['Proveedor'] != 'PEM 07']

# Filtrar los registros de df_remesas donde el proveedor es Clonación internacional
df_remesas_CI = df_remesas[df_remesas['Proveedor'] == 'Clonación internacional']
# Actualizar la columna 'Escenario' en df cuando las 'Llave' coinciden y el proveedor es PEM 07
df['Escenario'] = df.apply(
    lambda row: 'Clonación internacional' if row['Llave'] in df_remesas_CI['Llave'].values and row['Escenario'] not in ['Ya trabajados'] else row['Escenario'], axis=1)
# Eliminar los registros de df_remesas donde el proveedor es PEM 07
df_remesas = df_remesas[df_remesas['Proveedor'] != 'Clonación internacional']

In [18]:
# Escenario Se trabaja en archivo siguiente
df['Escenario'] = df['Escenario'].replace('', 'Se trabaja en archivo siguiente')

# Nuevas columnas para df 
df['Fecha trabajada'] = ''
df['Comentario y # remesa'] = ''
df['Proveedor'] = ''
df['Tipo Bloqueo'] = ''

# Reorganizar las columnas para colocar las nuevas después de 'Llave' y 'Escenario'
columnas_ordenadas = (
    df.columns.tolist()[:2]  # Primeras dos columnas (Llave y Escenario)
    + ['Fecha trabajada', 'Comentario y # remesa', 'Proveedor', 'Tipo Bloqueo']  # Nuevas columnas
    + df.columns.tolist()[2:-4]  # Resto de las columnas, excluyendo las 4 nuevas
)

# Reorganizar df con el nuevo orden
df = df[columnas_ordenadas]

# Eliminar columna de Marcador_Weconncet de df
df = df.drop('Marcador_Weconnect', axis=1)
# Eliminar columna de Llave tarj & Aut de df
df = df.drop('Llave tarj & Aut', axis=1)

In [19]:
# Fecha trabajada dia_hoy1
dia_hoy1 = datetime.now().strftime('%d/%m/%Y')

df['Fecha trabajada'] = df.apply(
    lambda row: dia_hoy1 if row['Escenario'] in ['Bloqueo R', 'Devolución Total', 'Enviados a remesa', 'ICE 5', 'Monto menor', 'PEM 07', 'Ticket prueba productiva', 'TNR', 'Weconnect', 'Clonación internacional', 'Aclaración Incorrecta', 'FNZ*ESC XHIDEL COMERCI'] else row['Fecha trabajada'], 
    axis=1
)

In [20]:
# Cruce de Comentario y # remesa de df_remesas
# Crear el diccionario de 'Comentario y # remesa' excluyendo las filas donde Escenario es "Ya trabajados"
comentario_dict = df_remesas[['Llave', 'Comentario y # remesa']].drop_duplicates().set_index('Llave')['Comentario y # remesa'].to_dict()

# Actualizar 'Comentario y # remesa' solo si Escenario no es "Ya trabajados"
df['Comentario y # remesa'] = df.apply(
    lambda row: comentario_dict.get(row['Llave'], row['Comentario y # remesa']) 
    if row['Escenario'] != 'Ya trabajados' else row['Comentario y # remesa'],
    axis=1
)

# Cruce de Proveedor de df_remesas a df
# Crear el diccionario de 'Proveedor' excluyendo las filas donde Escenario es "Ya trabajados"
proveedor_dict = df_remesas[['Llave', 'Proveedor']].drop_duplicates().set_index('Llave')['Proveedor'].to_dict()

# Actualizar 'Proveedor' solo si Escenario no es "Ya trabajados"
df['Proveedor'] = df.apply(
    lambda row: proveedor_dict.get(row['Llave'], row['Proveedor']) 
    if row['Escenario'] != 'Ya trabajados' else row['Proveedor'],
    axis=1
)

In [21]:
# Cruce de Comentario y # remesa de df_remesas a df
#comentario_dict = df_remesas[['Llave', 'Comentario y # remesa']].drop_duplicates().set_index('Llave')['Comentario y # remesa'].to_dict()

#df['Comentario y # remesa'] = df['Llave'].map(comentario_dict).combine_first(df['Comentario y # remesa'])

# Cruce de Proveedor de df_remesas a df
#proveedor_dict = df_remesas[['Llave', 'Proveedor']].drop_duplicates().set_index('Llave')['Proveedor'].to_dict()

#df['Proveedor'] = df['Llave'].map(proveedor_dict).combine_first(df['Proveedor'])

In [22]:
# Proveedor NO SE ENVIAN
df['Proveedor'] = df.apply(
    lambda row: 'NO SE ENVIAN' if row['Escenario'] in ['Bloqueo R', 'Devolución Total', 'ICE 5', 'Monto menor', 'PEM 07', 'Ticket prueba productiva', 'TNR', 'Weconnect', 'Clonación internacional', 'Primeras compras en agregadores', 'Primeras compras en agregadores', 'Aclaración Incorrecta', 'FNZ*ESC XHIDEL COMERCI'] else row['Proveedor'], 
    axis=1
)

# Comentario y # remesa es igual a Escenario si Proveedor es NO SE ENVIAN
df['Comentario y # remesa'] = df.apply(
    lambda row: row['Escenario'] if row['Proveedor'] == 'NO SE ENVIAN' else row['Comentario y # remesa'],
    axis=1
)

In [23]:
# Tipo Bloqueo es igual a TIPO_ACLARACIÓN si es distinto de Compra no reconocida o Primera compra CNP
df['Tipo Bloqueo'] = df.apply(
    lambda row: row['TIPO_ACLARACIÓN'] if row['Tipo Bloqueo'] == '' and row['Fecha trabajada'] != '' and pd.notna(row['Fecha trabajada']) and row['TIPO_ACLARACIÓN'] not in ['Compra no reconocida', 'Primera compra CNP'] else row['Tipo Bloqueo'],
    axis=1
)

# Tipo Bloqueo Fraude Digitado
df['Tipo Bloqueo'] = df.apply(
    lambda row: 'Fraude Digitado' if row['Fecha trabajada'] != '' and pd.notna(row['Fecha trabajada']) and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP'] and row['PEM'] in ['01', '10'] else row['Tipo Bloqueo'],
    axis=1
)

# Tipo Bloqueo Clonación
df['Tipo Bloqueo'] = df.apply(
    lambda row: 'Clonación' if row['Fecha trabajada'] != '' and pd.notna(row['Fecha trabajada']) and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP'] and row['PEM'] in ['02', '05', '07', '80', '90'] and row['Pais'] in ['MX'] else row['Tipo Bloqueo'],
    axis=1
)

# Tipo Bloqueo Clonación internacional
df['Tipo Bloqueo'] = df.apply(
    lambda row: 'Clonación internacional' if row['Fecha trabajada'] != '' and pd.notna(row['Fecha trabajada']) and row['TIPO_ACLARACIÓN'] in ['Compra no reconocida', 'Primera compra CNP'] and row['PEM'] in ['02', '05', '07', '80', '90'] and row['Pais'] not in ['MX'] else row['Tipo Bloqueo'],
    axis=1
)

In [24]:
# Eliminar filas duplicadas basándose en la columna 'Llave', conservando solo la primera ocurrencia
df_remesas = df_remesas.drop_duplicates(subset='Llave', keep='first').reset_index(drop=True)

In [25]:
# Filtrar las filas donde Proveedor es igual a 'BRADESCARD'
df_bradescard = df_remesas[
    df_remesas['Proveedor'] == 'BRADESCARD'
][[
    'TARJETA', 'COMENTARIO_TKT', 'REFERENCIA', 'TIPO_BLOQUEO', 
    'TICKET', 'FECHA_TICKET', 'MCC', 'COD_PAIS', 'IMPORTE', 
    'F_TRX', 'AUT', 'PEM', 'NOMBRE_DEL_COMERCIO', 'MERCHANT_CITY', 
    'AFILIACION', 'ICE', 'TOKEN_Q2', 'CLIENTE'
]]

# Filtrar las filas donde Proveedor es igual a 'BRADESCARD'
df_copayment = df_remesas[
    df_remesas['Proveedor'] == 'COPAYMENT'
][[
    'TARJETA', 'COMENTARIO_TKT', 'REFERENCIA', 'TIPO_BLOQUEO', 
    'TICKET', 'FECHA_TICKET', 'MCC', 'COD_PAIS', 'IMPORTE', 
    'F_TRX', 'AUT', 'PEM', 'NOMBRE_DEL_COMERCIO', 'MERCHANT_CITY', 
    'AFILIACION', 'ICE', 'TOKEN_Q2', 'CLIENTE'
]]

# Filtrar las filas donde Proveedor es igual a 'BRADESCARD'
df_prosa = df_remesas[
    df_remesas['Proveedor'] == 'PROSA'
][[
    'TARJETA', 'COMENTARIO_TKT', 'REFERENCIA', 'TIPO_BLOQUEO', 
    'TICKET', 'FECHA_TICKET', 'MCC', 'COD_PAIS', 'IMPORTE', 
    'F_TRX', 'AUT', 'PEM', 'NOMBRE_DEL_COMERCIO', 'MERCHANT_CITY', 
    'AFILIACION', 'ICE', 'TOKEN_Q2', 'CLIENTE'
]]

In [26]:
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle

# Paso 1: Generar el archivo principal
df.to_excel(path_archivo, index=False)

# Exportar inicialmente df_remesas a una hoja separada
with pd.ExcelWriter(path_archivo, engine='openpyxl', mode='a') as writer:
    df_remesas.to_excel(writer, sheet_name='Remesa', index=False)

print(f"El archivo principal '{pre_remesa}' ha sido creado. Realiza los cambios necesarios y guarda el archivo.")
print('Recuerda realizar las VALIDACIONES que se detallan MÁS ABAJO')
input("Presiona Enter cuando hayas terminado de realizar los cambios en el archivo principal...")

# Paso 2: Leer el archivo principal modificado
df = pd.read_excel(path_archivo, dtype=str)
df_remesas = pd.read_excel(path_archivo, sheet_name='Remesa', dtype=str)

# Si necesitas actualizar otros DataFrames basados en los cambios realizados:
df_bradescard = df_remesas[df_remesas['Proveedor'] == 'BRADESCARD']
df_copayment = df_remesas[df_remesas['Proveedor'] == 'COPAYMENT']
df_prosa = df_remesas[df_remesas['Proveedor'] == 'PROSA']

# Filtrar las filas donde Proveedor es igual a los valores correspondientes
columnas_seleccionadas = [
    'TARJETA', 'COMENTARIO_TKT', 'REFERENCIA', 'TIPO_BLOQUEO', 
    'TICKET', 'FECHA_TICKET', 'MCC', 'COD_PAIS', 'IMPORTE', 
    'F_TRX', 'AUT', 'PEM', 'NOMBRE_DEL_COMERCIO', 'MERCHANT_CITY', 
    'AFILIACION', 'ICE', 'TOKEN_Q2', 'CLIENTE'
]

df_bradescard = df_bradescard[columnas_seleccionadas]
df_copayment = df_copayment[columnas_seleccionadas]
df_prosa = df_prosa[columnas_seleccionadas]

# Paso 3: Crear nuevas hojas en el archivo principal
with pd.ExcelWriter(path_archivo, engine='openpyxl', mode='a') as writer:
    df_bradescard.to_excel(writer, sheet_name=f'Lote de aclaraciones {num_bradescard}', index=False)
    df_copayment.to_excel(writer, sheet_name=num_copayment, index=False)
    df_prosa.to_excel(writer, sheet_name=num_prosa, index=False)

# Aplicar formato de texto a las hojas generadas
wb = load_workbook(path_archivo)

# Definir estilo de texto
text_style = NamedStyle(name="text", number_format="@")

for sheet_name in [f'Lote de aclaraciones {num_bradescard}', num_copayment, num_prosa]:
    ws = wb[sheet_name]
    for row in ws.iter_rows():
        for cell in row:
            cell.style = text_style  # Aplicar formato de texto

wb.save(path_archivo)

# Crear archivos separados para cada proveedor y aplicar formato texto
for archivo, df_separado in zip([path_archivo_b, path_archivo_c, path_archivo_p], 
                                 [df_bradescard, df_copayment, df_prosa]):
    with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
        df_separado.to_excel(writer, index=False)
    
    # Aplicar formato texto a cada archivo
    wb = load_workbook(archivo)
    ws = wb.active
    for row in ws.iter_rows():
        for cell in row:
            cell.style = text_style  # Aplicar formato de texto
    wb.save(archivo)

print("")
print("Los archivos adicionales y las hojas en el archivo principal han sido generados correctamente.")

El archivo principal 'Pre remesa 15102025.xlsx' ha sido creado. Realiza los cambios necesarios y guarda el archivo.
Recuerda realizar las VALIDACIONES que se detallan MÁS ABAJO
Presiona Enter cuando hayas terminado de realizar los cambios en el archivo principal...

Los archivos adicionales y las hojas en el archivo principal han sido generados correctamente.



**VALIDAR:**

1.ATM (disposición de efectivo) que empiece por vuXXXX se envia PROSA tipo de bloqueo disposición de efectivo

2.Duplicados

3.BRADESCARD Fraudes Digitados solo mayores a 50 si son mx

4.PEM NULL en Tipo Bloqueo

5.Texto en columnas para fecha trabajada